# Cleaning a PostgreSQL Database
![Clean PostgreSQL Database](Project_Image.jpeg)

In this project, you will work with data from a hypothetical Super Store to challenge and enhance your SQL skills in data cleaning. This project will engage you in identifying top categories based on the highest profit margins and detecting missing values, utilizing your comprehensive knowledge of SQL concepts.

## Data Dictionary:

### `orders`:
| Column | Definition | Data type | Comments |
|--------|------------|-----------|----------|
| `row_id`| Unique Record ID | `INTEGER` |
| `order_id` | Identifier for each order in table | `TEXT` | Connects to `order_id` in `returned_orders` table |
| `order_date` | Date when order was placed | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |
| `region` | Region Customer belongs to | `TEXT` | Connects to `region` in `people` table |
| `product_id` | Identifier of Product bought | `TEXT` | Connects to `product_id` in `products` table |
| `sales` | Total Sales Amount for the Line Item | `DOUBLE PRECISION` |
| `quantity` | Total Quantity for the Line Item | `DOUBLE PRECISION` |
| `discount` | Discount applied for the Line Item | `DOUBLE PRECISION` |
| `profit` | Total Profit earned on the Line Item | `DOUBLE PRECISION` |

### `returned_orders`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `returned`| Yes values for Order / Line Item Returned | `TEXT` |
| `order_id` | Identifier for each order in table | `TEXT` |
| `market` | Market order_id belongs to | `TEXT` |

### `people`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `person`| Name of Salesperson credited with Order | `TEXT` |
| `region` | Region Salesperson in operating in | `TEXT` |

### `products`:
| Column | Definition | Data type |
|--------|------------|-----------|
| `product_id`| Unique Identifier for the Product | `TEXT` |
| `category` | Category Product belongs to | `TEXT` |
| `sub_category` | Sub Category Product belongs to | `TEXT` |
| `product_name` | Detailed Name of the Product | `TEXT` |

As you can see in the Data Dictionary above, date fields have been written to the `orders` table as `TEXT` and numeric fields like sales, profit, etc. have been written to the `orders` table as `Double Precision`. You will need to take care of these types in some of the queries. This project is an excellent opportunity to apply your SQL skills in a practical setting and gain valuable experience in data cleaning and analysis. Good luck, and happy querying!

Let's dive into the world of SQL and work our magic to clean and analyze data from our hypothetical Super Store.

---

## Task 1: Find the top 5 products from each category based on highest total sales

The output should be sorted by category in ascending order and by sales in descending order within each category, i.e., within each category, the product with highest margin should sit at the top.

**Save the query as `top_five_products_each_category`, containing the following columns:**

- `category`
- `product_name`
- `product_total_sales` (rounded to two decimal places)
- `product_total_profit` (rounded to two decimal places)
- `product_rank`

---

## Task 2: Impute missing quantity values

Calculate the quantity for orders with missing values in the `quantity` column by determining the unit price for each `product_id` using available order data, considering relevant pricing factors such as discount, market, or region. Then, use this unit price to estimate the missing quantity values. The calculated values should be stored in the `calculated_quantity` column.

**Save query output as `impute_missing_values`, containing the following columns:**

- `product_id`
- `discount`
- `market`
- `region`
- `sales`
- `quantity`
- `calculated_quantity` (rounded to zero decimal places)

In [12]:
import duckdb as db
import pandas as pd

In [14]:
orders = pd.read_csv("orders.csv")
returned_orders = pd.read_csv("returned_orders.csv")
people = pd.read_csv("people.csv")
products = pd.read_csv("products.csv")

In [18]:
# top_five_products_each_category
result = db.query("""
   SELECT * FROM (
SELECT products.category, 
	products.product_name,
  	ROUND(SUM(CAST(ord.sales AS NUMERIC)), 2) AS product_total_sales,
	ROUND(SUM(CAST(ord.profit AS NUMERIC)), 2) AS product_total_profit,
	RANK() OVER(PARTITION BY products.category ORDER BY SUM(ord.sales) DESC) AS product_rank
	FROM orders AS ord
	INNER JOIN products
		ON ord.product_id = products.product_id
	GROUP BY products.category, products.product_name
) AS tmp
WHERE product_rank < 6; 
""").to_df()

result

,category,product_name,product_total_sales,product_total_profit,product_rank
0,Technology,"Apple Smart Phone, Full Size",86935.78,5921.58,1
1,Technology,"Cisco Smart Phone, Full Size",76441.53,17238.52,2
2,Technology,"Motorola Smart Phone, Full Size",73156.30,17027.11,3
3,Technology,"Nokia Smart Phone, Full Size",71904.56,9938.19,4
4,Technology,Canon imageCLASS 2200 Advanced Copier,61599.82,25199.93,5
5,Office Supplies,"Eldon File Cart, Single Width",39873.23,5571.26,1
6,Office Supplies,"Hoover Stove, White",32842.61,-2180.63,2
7,Office Supplies,"Hoover Stove, Red",32644.13,11651.68,3
8,Office Supplies,"Rogers File Cart, Single Width",29558.82,2368.82,4
9,Office Supplies,"Smead Lockers, Industrial",28991.66,3630.44,5


In [24]:
# impute_missing_values
result = db.query("""
    WITH missing AS (
	SELECT product_id,
		discount, 
		market,
		region,
		sales,
		quantity
	FROM orders 
	WHERE quantity IS NULL
), 

unit_prices AS (SELECT o.product_id,
	CAST(o.sales / o.quantity AS NUMERIC) AS unit_price
FROM orders o
RIGHT JOIN missing AS m 
	ON o.product_id = m.product_id
	AND o.discount = m.discount
WHERE o.quantity IS NOT NULL
)

SELECT DISTINCT m.*,
	ROUND(CAST(m.sales AS NUMERIC) / up.unit_price,0) AS calculated_quantity
FROM missing AS m
INNER JOIN unit_prices AS up
	ON m.product_id = up.product_id;
""")

result

┌──────────────────┬──────────┬─────────┬─────────┬─────────┬──────────┬─────────────────────┐
│    product_id    │ discount │ market  │ region  │  sales  │ quantity │ calculated_quantity │
│     varchar      │  double  │ varchar │ varchar │ double  │  double  │       double        │
├──────────────────┼──────────┼─────────┼─────────┼─────────┼──────────┼─────────────────────┤
│ FUR-ADV-10004395 │      0.0 │ EMEA    │ EMEA    │   84.12 │     NULL │                 2.0 │
│ FUR-ADV-10000571 │      0.0 │ EMEA    │ EMEA    │  438.96 │     NULL │                 4.0 │
│ TEC-STA-10004542 │      0.0 │ Africa  │ Africa  │  160.32 │     NULL │                 4.0 │
│ TEC-STA-10003330 │      0.0 │ Africa  │ Africa  │  506.64 │     NULL │                 2.0 │
│ FUR-BO-10001337  │     0.15 │ US      │ West    │ 308.499 │     NULL │                 3.0 │
└──────────────────┴──────────┴─────────┴─────────┴─────────┴──────────┴─────────────────────┘